## Data overview ##

We will explore a few different data sets that are all publicly available:

* Perseus from COMPLETE ([Ridge et al. 2006](https://ui.adsabs.harvard.edu/abs/2006AJ....131.2921R/abstract)) [DATA LINK](https://lweb.cfa.harvard.edu/COMPLETE/data_html_pages/FCRAO.html)
* Ophiuchus from COMPLETE ([Ridge et al. 2006](https://ui.adsabs.harvard.edu/abs/2006AJ....131.2921R/abstract)) [DATA LINK](https://lweb.cfa.harvard.edu/COMPLETE/data_html_pages/FCRAO.html)
* Orion from CARMA-NRO ([Kong et al. 2018](https://ui.adsabs.harvard.edu/abs/2018ApJS..236...25K/abstract); [Kong et al. 2021](https://ui.adsabs.harvard.edu/abs/2021RNAAS...5...55K/abstract)) [DATA LINK](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6Q26PN)

In this tutorial, we show the preprocessing steps applied to the data from the public data repositories linked above.

Homogenizing different datasets before making comparisons is a common practice. In this notebook, we also highlight how to reduce the data volume after matching to a common spatial resolution and pixel grid size.

**Note that the final steps of spatially convoling and downsampling the Orion CARMA-NRO data will take hours to run and require >4 GB of memory. To avoid this, download the data from Eric's google drive link**


In [ ]:
# Enable plotting in the notebook

%matplotlib inline

In [ ]:
# Define the path to the data, whereever you downloaded it to:

import os
import numpy as np

data_path = "/Users/ekoch/storage/ISM_School_LocalClouds/"


## Ophiuchus data: Trimming the empty regions ##



In [ ]:
from astropy.io import fits
from spectral_cube import SpectralCube, Projection

ophA_12co_mom0 = Projection.from_hdu(fits.open(os.path.join(data_path, "orig/Ophiuchus/OphA_12coFCRAO_F_map.fits")))


Let's plot the data to have a quicklook at it:

In [ ]:
ophA_12co_mom0.quicklook(use_aplpy=False)

The methods used to calculate many turbulence statistics are affected by large empty regions in the maps (for example, the power spectrum). To minimize empty space, we will slice the Ophiuchus data down to just the large contiguous map.

To slice out the bigger region, we will identify regions with finite values using `scipy.ndimage`

In [ ]:
import scipy.ndimage as nd

finite_mask = np.isfinite(ophA_12co_mom0)

list_of_slices = nd.find_objects(nd.label(finite_mask)[0])

# The first slice should correspond to the largest region:
this_slice = list_of_slices[0]

print(this_slice)

We now apply this to the data, make an image to ensure we have the right region, and save the sliced image as a new FITS file.

In [ ]:
ophA_12co_mom0_sliced = ophA_12co_mom0[this_slice]

ophA_12co_mom0_sliced.quicklook(use_aplpy=True)

ophA_12co_mom0_sliced.write(os.path.join(data_path, "Ophiuchus/OphA_12coFCRAO_F_map_sliced.fits"))

Perfect! We have exactly sliced out the largest contiguous region in the map.

Note that we have removed the smaller region from the map. You could save that region to its own file by selecting the _second_ element in `list_of_slices` above.

Now we need to repeat this procedure for the 12CO data cube, and then for the 13CO map and data cube:

In [ ]:
cube_12co = SpectralCube.read(os.path.join(data_path, "orig/Ophiuchus/OphA_12coFCRAO_F_vxy.fits"))

# Make sure the map and cube have the same spatial pixel dimensions
# The 0th axis in cube_12co is the velocity axis.
assert ophA_12co_mom0.shape == cube_12co.shape[1:]

# The cube also has a spectral axis. By adding a slice(None) to this_slice, we avoid slicing along the spectral axis in the cube.
cube_12co_sliced = cube_12co[(slice(None),) + this_slice]

# An intermediate step can be saved, but see below for adding information about the telescope beam.
# cube_12co_sliced.write(os.path.join(data_path, "Ophiuchus/OphA_12coFCRAO_F_vxy_sliced.fits"))


In [ ]:
# Same as above, condensed down for the 13CO:
ophA_13co_mom0 = Projection.from_hdu(fits.open(os.path.join(data_path, "orig/Ophiuchus/OphA_13coFCRAO_F_map.fits")))

finite_mask = np.isfinite(ophA_13co_mom0)

list_of_slices = nd.find_objects(nd.label(finite_mask)[0])

# The first slice should correspond to the largest region:
this_slice = list_of_slices[0]

ophA_13co_mom0_sliced = ophA_13co_mom0[this_slice]

ophA_13co_mom0_sliced.quicklook(use_aplpy=True)

# An intermediate step can be saved, but see below for adding information about the telescope beam.

# ophA_13co_mom0_sliced.write(os.path.join(data_path, "Ophiuchus/OphA_13coFCRAO_F_map_sliced.fits"), overwrite=True)


cube_13co = SpectralCube.read(os.path.join(data_path, "orig/Ophiuchus/OphA_13coFCRAO_F_vxy.fits"))

# Make sure the map and cube have the same spatial pixel dimensions
# The 0th axis in cube_13co is the velocity axis.
assert ophA_13co_mom0.shape == cube_13co.shape[1:]

# The cube also has a spectral axis. By adding a slice(None) to this_slice, we avoid slicing along the spectral axis in the cube.
cube_13co_sliced = cube_13co[(slice(None),) + this_slice]

# cube_13co_sliced.write(os.path.join(data_path, "Ophiuchus/OphA_13coFCRAO_F_vxy_sliced.fits"), overwrite=True)

## COMPLETE data: appending beam information to the FITS headers ##

The COMPLETE data site clearly states that the beam size of the data is 46" (FWHM). However, the FITS headers don't contain that information and it can be very handy to include.

We will include the beam information by using the `radio_beam` package.

In [ ]:
from radio_beam import Beam
import astropy.units as u

complete_beam = Beam(major=46 * u.arcsec)  # By giving the major axis, we assume the beam is circular.

complete_beam

Now we will apply this to the Ophichus data (since we already loaded it above), and then the Perseus data.

In [ ]:
ophA_12co_mom0_sliced_wbeam = ophA_12co_mom0_sliced.with_beam(complete_beam)
ophA_12co_mom0_sliced_wbeam.write(os.path.join(data_path, "Ophiuchus/OphA_12coFCRAO_F_map_sliced_withbeam.fits"), overwrite=True)

cube_12co_sliced_wbeam = cube_12co_sliced.with_beam(complete_beam)


## COMPLETE data: slicing out the relevant velocity range ##

To reduce the data volume, we can also limit the number of empty (signal-free) velocity channels. For Ophiuchus, the vast majority of the signal is between 0 and 8 km/s.

The velocity channels are also moderately correlated to their nearest neighbours. Because of that, we will also downsample along the spectral axies using the `SpectralCube.downsample_axis` function, which averages the adjacent channels together.

In [ ]:
cube_12co_sliced_wbeam = cube_12co_sliced_wbeam.spectral_slab(0 * u.km / u.s, 8 * u.km / u.s)
cube_12co_sliced_wbeam = cube_12co_sliced_wbeam.downsample_axis(axis=0, factor=2)

cube_12co_sliced_wbeam.write(os.path.join(data_path, "Ophiuchus/OphA_12coFCRAO_F_vxy_sliced_withbeam.fits"), overwrite=True)

In [ ]:
# Repeat for 13CO, just condensed.

ophA_13co_mom0_sliced_wbeam = ophA_13co_mom0_sliced.with_beam(complete_beam)
ophA_13co_mom0_sliced_wbeam.write(os.path.join(data_path, "Ophiuchus/OphA_13coFCRAO_F_map_sliced_withbeam.fits"), overwrite=True)

cube_13co_sliced_wbeam = cube_13co_sliced.with_beam(complete_beam)
cube_13co_sliced_wbeam = cube_13co_sliced_wbeam.spectral_slab(0 * u.km / u.s, 8 * u.km / u.s)
cube_13co_sliced_wbeam = cube_13co_sliced_wbeam.downsample_axis(axis=0, factor=2)
cube_13co_sliced_wbeam.write(os.path.join(data_path, "Ophiuchus/OphA_13coFCRAO_F_vxy_sliced_withbeam.fits"), overwrite=True)


### And now the Perseus data. ###

Note that we did not slice a region of Perseus because the map already has less empty areas.

Below we:

1. Append the beam information
2. Slice out the relevant velocity range: -5 to 15 km/s.
3. Downsample along the spectral axis by a factor of 2.

In [ ]:

perA_12co_mom0 = Projection.from_hdu(fits.open(os.path.join(data_path, "orig/Perseus/PerA_12coFCRAO_F_map.fits")))
perA_12co_mom0_sliced_wbeam = perA_12co_mom0.with_beam(complete_beam)
perA_12co_mom0_sliced_wbeam.write(os.path.join(data_path, "Perseus/PerA_12coFCRAO_F_map_sliced_withbeam.fits"), overwrite=True)

perA_cube_12co = SpectralCube.read(os.path.join(data_path, "orig/Perseus/perA_12coFCRAO_F_vxy.fits"))
perA_cube_12co.allow_huge_operations = True
perA_cube_12co_sliced_wbeam = perA_cube_12co.with_beam(complete_beam)
perA_cube_12co_sliced_wbeam = perA_cube_12co_sliced_wbeam.spectral_slab(-5 * u.km / u.s, 15 * u.km / u.s)
perA_cube_12co_sliced_wbeam = perA_cube_12co_sliced_wbeam.downsample_axis(axis=0, factor=2)
perA_cube_12co_sliced_wbeam.write(os.path.join(data_path, "Perseus/PerA_12coFCRAO_F_vxy_sliced_withbeam.fits"), overwrite=True)


In [ ]:
# Checkout the integrated intensity map of Perseus.
perA_12co_mom0.quicklook(use_aplpy=True)

In [ ]:
# Repeat for 13CO, condensed into 1 cell.

perA_13co_mom0 = Projection.from_hdu(fits.open(os.path.join(data_path, "orig/Perseus/PerA_13coFCRAO_F_map.fits")))
perA_13co_mom0_sliced_wbeam = perA_13co_mom0.with_beam(complete_beam)
perA_13co_mom0_sliced_wbeam.write(os.path.join(data_path, "Perseus/PerA_13coFCRAO_F_map_sliced_withbeam.fits"), overwrite=True)

perA_cube_13co = SpectralCube.read(os.path.join(data_path, "orig/Perseus/perA_13coFCRAO_F_vxy.fits"))
perA_cube_13co.allow_huge_operations = True
perA_cube_13co_sliced_wbeam = perA_cube_13co.with_beam(complete_beam)
perA_cube_13co_sliced_wbeam = perA_cube_13co_sliced_wbeam.spectral_slab(-5 * u.km / u.s, 15 * u.km / u.s)
perA_cube_13co_sliced_wbeam = perA_cube_13co_sliced_wbeam.downsample_axis(axis=0, factor=2)
perA_cube_13co_sliced_wbeam.write(os.path.join(data_path, "Perseus/PerA_13coFCRAO_F_vxy_sliced_withbeam.fits"), overwrite=True)


perA_13co_mom0.quicklook(use_aplpy=True)

## CARMA-NRO data: smoothing to match the COMPLETE resolution and trimming a redundant axis ##

The CARMA-NRO Orion maps show incredible detail. However, when comparing to other data sets, it is often useful to homogenize the data. In this case, the key difference is matching the spatial resolution of the COMPLETE data (46"). To do this, we will load the CARMA-NRO data at the original ~10" and convolve to the same resolution using spectral-cube.

In [ ]:
# We already have the COMPLETE beam defined above. Just a reminder: it's a 46" FWHM beam.
complete_beam

The Orion maps retain a redundant spectral axis (i.e., a dimension of size 1). To load the moment 0 maps as a 2D image, we will first trim out by loading it as a "cube" and slicing along the spatial dimension:

In [ ]:
orion_mom0_12co = SpectralCube.read(os.path.join(data_path, "orig/Orion/mom0_12co_pix_2_Tmb.fits"))[0] # Slice out the 0th image

orion_mom0_12co.quicklook(use_aplpy=True)

The original beam size is:

In [ ]:
orion_mom0_12co.beam

To match the COMPLETE beam size, we will use `convolve_to`, which uses the `astropy.convolution.convolve_fft` function to smooth the data.

In [ ]:
orion_mom0_12co_smoothed = orion_mom0_12co.convolve_to(complete_beam, allow_huge=True)

In [ ]:
orion_mom0_12co_smoothed.quicklook(use_aplpy=True)

The Orion data now have the same _angular_ resolution as the COMPLETE data. Note that we have emphasized matching the angular resolution than physical scale. On small scales, the pixel-to-pixel correlation is dominated by the telescope's beam, and so we are roughly matching _observational_ conditions with the angular resolution.

We will save this smoothed version:

Now we will repeat for the other moment 0 maps and the cubes.

### *NOTE: the full cubes are ~2 GB each. Convolving the data requires loading a 2nd version into memory. This will require >4 GB of RAM.* ###

# Convolving the cubes below. #

**WARNING: this will take >2 hr to run and will require >4 GB of RAM to load the data**

We will now repeat this for every channel in the different data cubes.

There are two steps performed below:

1. Convolution of the data to a 46" beam using `SpectralCube.convolve_to`
2. Downsampling the pixel size to have ~2 pixels per FWHM. This is done using the `SpectralCube.downsample_axis` function, applied separately along both spatial axes (so `axis=1` and `axis=2` in the cubes).

For 2., the downsampling is done over a factor of 11. This factors reduces each spatial pixel from the original 2" to 22", approximately ~2 pixels per 46" FWHM beam. This is also roughly the factor in the COMPLETE data. Most notably, this drastically reduces the data volume of the spatially-matched cubes.


In [ ]:
orion_cube_12co = SpectralCube.read(os.path.join(data_path, "orig/Orion/CARMA_NRO_Orion_12co.fits"))

# Tell spectral-cube it is OK to load the whole cube into memory:
orion_cube_12co.allow_huge_operations = True

orion_cube_12co_smoothed = orion_cube_12co.convolve_to(complete_beam, allow_huge=True)

# Downsample to have ~2 pixels per FWHM. This could be done through reprojection, but we can also average over each
# spatial dimension to downsample the data:
orion_cube_12co_downsamp = orion_cube_12co_smoothed.downsample_axis(axis=1, factor=11).downsample_axis(axis=2, factor=11)

orion_cube_12co_downsamp.write(os.path.join(data_path, "Orion/CARMA_NRO_Orion_12co_smoothed46.fits"), overwrite=True)

We also want to reduce the spatial pixel sizes of the smoothed moment 0 maps. However, the `downsample_axis` function is not currently implemented for 2D images in spectral-cube. Instead, we can use the downsampled cube's FITS header and reproject the data to a matched spatial grid using `Projection.reproject` (which uses the [reproject](https://github.com/astropy/reproject) package).

In [ ]:
orion_mom0_12co_smoothed = orion_mom0_12co_smoothed.reproject(orion_cube_12co_downsamp[0].header)

orion_mom0_12co_smoothed.write(os.path.join(data_path, "Orion/mom0_12co_pix_2_Tmb_smoothed46.fits"), overwrite=True)

The following is the same operation applied to the 13CO and C18O data:

In [ ]:
orion_cube_13co = SpectralCube.read(os.path.join(data_path, "orig/Orion/CARMA_NRO_Orion_13co.fits"))

# Tell spectral-cube it is OK to load the whole cube into memory:
orion_cube_13co.allow_huge_operations = True

orion_cube_13co_smoothed = orion_cube_13co.convolve_to(complete_beam, allow_huge=True)

orion_cube_13co_downsamp = orion_cube_13co_smoothed.downsample_axis(axis=1, factor=11).downsample_axis(axis=2, factor=11)

orion_cube_13co_downsamp.write(os.path.join(data_path, "Orion/CARMA_NRO_Orion_13co_smoothed46.fits"), overwrite=True)

In [ ]:
orion_mom0_13co = SpectralCube.read(os.path.join(data_path, "orig/Orion/mom0_13co_pix_2_Tmb.fits"))[0] # Slice out the 0th image

orion_mom0_13co_smoothed = orion_mom0_13co.convolve_to(complete_beam, allow_huge=True)
orion_mom0_13co_smoothed = orion_mom0_13co_smoothed.reproject(orion_cube_13co_downsamp[0].header)

orion_mom0_13co_smoothed.write(os.path.join(data_path, "Orion/mom0_13co_pix_2_Tmb_smoothed46.fits"), overwrite=True)

In [ ]:
orion_cube_c18o = SpectralCube.read(os.path.join(data_path, "orig/Orion/CARMA_NRO_Orion_c18o.fits"))

# Tell spectral-cube it is OK to load the whole cube into memory:
orion_cube_c18o.allow_huge_operations = True

orion_cube_c18o_smoothed = orion_cube_c18o.convolve_to(complete_beam, allow_huge=True)

orion_cube_c18o_downsamp = orion_cube_c18o_smoothed.downsample_axis(axis=1, factor=11).downsample_axis(axis=2, factor=11)

orion_cube_c18o_downsamp.write(os.path.join(data_path, "Orion/CARMA_NRO_Orion_c18o_smoothed46.fits"), overwrite=True)

In [ ]:
orion_mom0_c18o = SpectralCube.read(os.path.join(data_path, "orig/Orion/mom0_c18o_pix_2_Tmb.fits"))[0] # Slice out the 0th image

orion_mom0_c18o_smoothed = orion_mom0_c18o.convolve_to(complete_beam, allow_huge=True)
orion_mom0_c18o_smoothed = orion_mom0_c18o_smoothed.reproject(orion_cube_c18o_downsamp[0].header)

orion_mom0_c18o_smoothed.write(os.path.join(data_path, "Orion/mom0_c18o_pix_2_Tmb_smoothed46.fits"), overwrite=True)